# Radio flux density calibration of Hydra A @ 5 GHz


### Please note:

The data you will be processing in this and other tutorials is located in the "data" folder. In there you will
find fits files for your calibrator [Hydra A](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=NAME%20HYDRA%20A)
and your target source [J1427-4206](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=PKS+1424-418&submit=SIMBAD+search) at 2 GHz, 5 GHz, 8 GHz and 12 GHz frequencies. **For this exercize we will be working on
5GHz data for our calibration source**.

All the files you will need for the tutorial can be found at this [Github repository](https://github.com/Pfesi/AVN2019). **You need to download the file and work from that directory**. i.e. "cd into that directory"

# Background 

Source calibration is the same for all sources so we will not repeat everything here. Notes will be made along the way where differences occur.




Listed below are the steps we are going to follow to calibrate our source

# <a name="top"></a>Task list:

1. [Locate and open the observation file](#section1)
2. [Examine the fits file](#section2)
3. [Extract the drift scan data](#section3)
4. [Measure the antenna temperature](#section5)
 * 4.1 [Convert counts to units of Kelvin](#section5.1)
 * 4.2 [Fit a baseline to remove systemic contributions](#section5.2)
 * 4.3 [Fit beam to get antenna temperature](#section5.3)
 * 4.4 [Amplitude correction](#section5.4)
5. [Calculate the point source sensitivity (PSS)](#section6)
6. [Got to target flux](5GHz_j1427-4206_tutorial.ipynb)

# 1. <a name="section1">Locate and open the observation file</a> 
[back to top](#top)

Select an observation to work on and open the fits file





In [ ]:
import astropy.io.fits as pyfits

fitsfile = 'data/HydraA/5GHz/2013d125_15h35m54s_Cont_george_HYDRA_A.fits'
hdulist  = pyfits.open(fitsfile) 

# 2. <a name="section2">Examine the fits file</a>
[back to top](#top)

It is always a good idea to familiarize yourself with the file contents by viewing the different types of information inside your file. To get an overview of the contents we use the info() method


In [ ]:
hdulist.info()

The info() method gives us a list of all the HDUs in this file. 

* The first header data unit (HDU) contains information on the  observation.  
* The second one has information on the 2.5 cm (i.e. 12 GHz) feed system. This data is not necessarily up to date, so we generally disregard it. 
* The third unit has the noise diode firing, used to convert from raw counts to Kelvins. 
* The next three binary tables are the drift scans themselves, starting at the north offset position.
* The last HDU contains more information about the observation.

To view the content inside an HDU we use indexes. For example, to view the PRIMARY HDU, we use index 0.


In [ ]:
hdulist[0].header

# You can play around with changing the indeces to see what 
# other information is stored in the other HDUs

# 3. <a name="section3">Extract the drift scan data</a>

[back to top](#top)

The 26-m telescope has slight variations in pointing, so in order to make sure we accurately measure the peak flux of the source, we do a series of drift scans at the expected north and south half-power points, as well as the centre. 
Basically we park the telescope at a coordinate just ahead of where the source currently is in the sky, and wait for earth's rotation to carry the telescope beam across the source. The advantage of this is that you don't have to worry about whether your telescope can drive at a consistent speed across the source. The 26-m tracks perfectly at sidereal rate, but is not very good at higher speeds. Other telescopes may choose to use on-the-fly scans, where they sample the power at a high rate while slewing across the source.



In [ ]:
drift1 = hdulist[3] #HPN
drift2 = hdulist[4] #ON
drift3 = hdulist[5] #HPS

Lets view the drift1 HDU to see the file contents

In [ ]:
drift1.header

Most of the fields are actually just the position in different formats. We are interested in the output signal '**Count1**' and '**Count2**'.  

# 4. <a name="section5">Measure the antenna temperature</a>
[back to top](#top)

## 4.1 <a name="section5.1">Convert counts to units of Kelvin </a>
[back to top](#top)

In [ ]:
#Import plotting libraries
import matplotlib.pyplot as plt

#display figures inline 
%matplotlib inline  

# Get the noise diode calibration HDU
noise_cal = hdulist[2]

# Construct an array for the x-axis in terms of right ascension
ra = drift1.data['RA_J2000']

count_K1 = noise_cal.header['HZPERK1']
count_K2 = noise_cal.header['HZPERK2']

# Get source counts and convert counts to antenna temperature
north_scan_LCP = (drift1.data['Count1']/count_K1) - ((drift1.data['Count1'])[0]/count_K1) 
north_scan_RCP = (drift1.data['Count2']/count_K2) - ((drift1.data['Count2'])[0]/count_K2)
on_scan_LCP    = (drift2.data['Count1']/count_K1) - ((drift2.data['Count1'])[0]/count_K1)
on_scan_RCP    = (drift2.data['Count2']/count_K2) - ((drift2.data['Count2'])[0]/count_K2)
south_scan_LCP = (drift3.data['Count1']/count_K1) - ((drift3.data['Count1'])[0]/count_K1)
south_scan_RCP = (drift3.data['Count2']/count_K2) - ((drift3.data['Count2'])[0]/count_K2)
            
# Plot the data

# HPN 
plt.figure(figsize=[15,5])
plt.suptitle('North scan')

ax = plt.subplot(121)
plt.grid()
plt.plot(ra, north_scan_LCP, label = 'LCP')
plt.xlabel('RA (deg)')
plt.ylabel('Ta (K)')
plt.legend()
lim = plt.axis('tight')

ax = plt.subplot(122)
plt.grid()
plt.plot(ra, north_scan_RCP, label = 'RCP')
plt.xlabel('RA (deg)')
plt.ylabel('Ta (K)')
plt.legend()
lim = plt.axis('tight')

# ON
plt.figure(figsize=[15,5])
plt.suptitle('Central (on-source) scan')

ax = plt.subplot(121)
plt.grid()
plt.plot(ra, on_scan_LCP, label = 'LCP')
plt.xlabel('RA (deg)')
plt.ylabel('Ta (K)')
plt.legend()
lim = plt.axis('tight')

ax = plt.subplot(122)
plt.grid()
plt.plot(ra, on_scan_RCP, label = 'RCP')
plt.xlabel('RA (deg)')
plt.ylabel('Ta (K)')
plt.legend()
lim = plt.axis('tight')

# HPS
plt.figure(figsize=[15,5])
plt.suptitle('South scan')

ax = plt.subplot(121)
plt.grid()
plt.plot(ra, south_scan_LCP, label = 'LCP')
plt.xlabel('RA (deg)')
plt.ylabel('Ta (K)')
plt.legend()
lim = plt.axis('tight')

ax = plt.subplot(122)
plt.plot(ra, south_scan_RCP, label = 'RCP')
plt.xlabel('RA (deg)')
plt.ylabel('Ta (K)')
plt.legend(frameon=False)
plt.grid()
lim = plt.axis('tight')


We can clearly see the effect of the Dicke switching here, instead of seeing one smooth beam we see a combination of two sepearate beams. 



## 4.2 <a name="section5.2">Fit a baseline to remove systemic contributions</a>
[back to top](#top)


What do we fit then?  We can clealy see two antenna beam patterns for each scan.  The same process applies as for a single beam scan,  the only difference is that now we fit two beams instead of one.


Locate the section you want to use for your fit. We are going to write a few methods to help us process the location selection for our fit

In [ ]:
def getBasePts(x, y,len_scan,x1,x2,x3,x4): 
    '''
        Get baseline points. Select points along the driftscan where you will fit
        your baseline.
    '''
    xbleft = x[x1:x2]
    ybleft = y[x1:x2]
    xbright= x[len_scan-x3:len_scan-x4]
    ybright= y[len_scan-x3:len_scan-x4]

    left_base_pts = list(xbleft) + list(xbright)
    right_base_pts = list(ybleft) + list(ybright)

    return left_base_pts, right_base_pts 

Fit the baseline and plot your data

In [ ]:
import numpy as np

def fitBasePts(x_base_pts,y_base_pts, x, y):
    """Fit the data to remove systematic contributions.
    """
        
    base_fit_coeffs = np.polyfit(x_base_pts, y_base_pts, 1)
    base_fit_line = np.polyval(base_fit_coeffs,x_base_pts)   
    data_fit = np.polyval(base_fit_coeffs, x)
    data_fit = y - data_fit
    
    res,rms = residual(y_base_pts,base_fit_line)
   
    print ("Fit = %.2fx + (%d), rms error = %.3f" %(base_fit_coeffs[0], base_fit_coeffs[1],rms))
    
    return data_fit, base_fit_line
    
# Get the residual and rms to estimate the errors in the fit
def residual(model, data): 
    
    from sklearn.metrics import mean_squared_error
    """
        Calculate the residual between the model and the data.
    """

    res = np.array(model - data)
    rms = np.sqrt(mean_squared_error(data,model))

    return res, rms

def fitBaselineAndPlot(x,y,title,x1,x2,x3,x4,len_scan):
    """
        Fit the baseline and plot your data
    """


    xb,yb = getBasePts(x,y,len_scan,x1,x2,x3,x4)
    
    # Fit the baseline points
    fit, base_fit_line = fitBasePts(xb,yb, x, y)

    # plot the data
    plt.figure(figsize=[15,5])
    plt.suptitle(title)

    ax = plt.subplot(121)
    plt.axhline(y=0, color='k', alpha= 0.5)
    l1,= plt.plot(x,y, label = 'original data')
    l2,= plt.plot(xb,yb,'r.', label = 'noise')
    plt.plot(xb,base_fit_line, 'g', label = "baseline fit")
    plt.xlabel('RA (deg)')
    plt.ylabel('Ta (K)')
    plt.legend()
    plt.grid()
    lim = plt.axis('tight')

    ax = plt.subplot(122)
    l1,= plt.plot(x,fit, label = 'final data')
    plt.xlabel('RA (deg)')
    plt.ylabel('Ta (K)')
    plt.axhline(y=0, color='k', alpha= 0.5)
    plt.legend()
    plt.grid()
    lim = plt.axis('tight')
    
    return fit

# Get the length of the scans
len_scan = len(ra)

# Set the baseline points, default is 100 points on each side
# You can change the x1-x4 values to adjust the location of
# your baseline fit.

 # YOU CAN MODIFY THESE VALUES
x1 = 
x2 = 
x3 = 
x4 = 
    
print ("Y = mx + c")
# Fit and plot 
slcp  = fitBaselineAndPlot(ra, south_scan_LCP, 'South scan [LCP]',x1,x2,x3,x4,len_scan)
srcp  = fitBaselineAndPlot(ra, south_scan_RCP, 'South scan [RCP]',x1,x2,x3,x4,len_scan)
nlcp  = fitBaselineAndPlot(ra, north_scan_LCP, 'North scan [LCP]',x1,x2,x3,x4,len_scan)
nrcp  = fitBaselineAndPlot(ra, north_scan_RCP, 'Northscan [RCP]',x1,x2,x3,x4,len_scan)
onlcp = fitBaselineAndPlot(ra, on_scan_LCP, 'On scan [LCP]',x1,x2,x3,x4,len_scan)
onrcp = fitBaselineAndPlot(ra, on_scan_RCP, 'On scan [RCP]',x1,x2,x3,x4,len_scan)


## 4.3 <a name="section5.3">Fit beam to get antenna temperature</a>
[back to top](#top)

We are now going to fit the top of the beam to fit the 
antenna temperature. We will use a 2nd order polynomial fit
at the top of the peak. 

In [ ]:
# Get location of peak
def getAPeakPts(drift, percentage):
    """Get locations of scan peak where we are going to fit 
       We will be fitting the top 40% of our scan.
    """
    peak_frac = float((100.0 - percentage)/100.0)
    peak_max = max(drift)
    peak_pts = np.where(drift > peak_frac * peak_max)[0]
    return peak_pts

def getBPeakPts(drift, percentage):
    """Get locations of scan peak where we are going to fit 
       We will be fitting the top 40% of our scan.
    """
    peak_frac = float((100.0 - percentage)/100.0)
    peak_min = min(drift)
    peak_pts = np.where(drift < peak_frac * peak_min)[0]
    return peak_pts

# Fit the peak
def fitPeak(peak_pts, x, y):
    """Fit the peak and estimate errors.
    """

    peakfitcoeffs = np.polyfit(x[peak_pts], y[peak_pts], 2)
    peakfitline = np.polyval(peakfitcoeffs,x[peak_pts])
    res, rms = residual(peakfitline, y[peak_pts])

    return peakfitline,rms, x[peak_pts], y[peak_pts]

# Get peak points

# YOU CAN MODIFY THIS VALUE
peak_percentage = 40 # Change the percentage of the peak data you want to fit.

# Get peak points, Abeam
Apeak_pts_slcp  = getAPeakPts(slcp, peak_percentage)
Apeak_pts_srcp  = getAPeakPts(srcp, peak_percentage)
Apeak_pts_nlcp  = getAPeakPts(nlcp, peak_percentage)
Apeak_pts_nrcp  = getAPeakPts(nrcp, peak_percentage)
Apeak_pts_onlcp = getAPeakPts(onlcp, peak_percentage)
Apeak_pts_onrcp = getAPeakPts(onrcp, peak_percentage)

# Get peak points, Bbeam
Bpeak_pts_slcp  = getBPeakPts(slcp, peak_percentage)
Bpeak_pts_srcp  = getBPeakPts(srcp, peak_percentage)
Bpeak_pts_nlcp  = getBPeakPts(nlcp, peak_percentage)
Bpeak_pts_nrcp  = getBPeakPts(nrcp, peak_percentage)
Bpeak_pts_onlcp = getBPeakPts(onlcp, peak_percentage)
Bpeak_pts_onrcp = getBPeakPts(onrcp, peak_percentage)

# Fit the peaks
Afitpsl, Aslrms, Axpslcp, Aypslcp   = fitPeak(Apeak_pts_slcp, ra, slcp)
Afitpsr, Asrrms, Axpsrcp, Aypsrcp   = fitPeak(Apeak_pts_srcp, ra, srcp)
Afitpnl, Anlrms, Axpnlcp, Aypnlcp   = fitPeak(Apeak_pts_nlcp, ra, nlcp)
Afitpnr, Anrrms, Axpnrcp, Aypnrcp   = fitPeak(Apeak_pts_nrcp, ra, nrcp)
Afitponl, Aonlrms, Axpolcp, Aypolcp = fitPeak(Apeak_pts_onlcp, ra, onlcp)
Afitponr, Aonrrms, Axporcp, Ayporcp = fitPeak(Apeak_pts_onrcp, ra, onrcp)

Bfitpsl, Bslrms, Bxpslcp, Bypslcp   = fitPeak(Bpeak_pts_slcp, ra, slcp)
Bfitpsr, Bsrrms, Bxpsrcp, Bypsrcp   = fitPeak(Bpeak_pts_srcp, ra, srcp)
Bfitpnl, Bnlrms, Bxpnlcp, Bypnlcp   = fitPeak(Bpeak_pts_nlcp, ra, nlcp)
Bfitpnr, Bnrrms, Bxpnrcp, Bypnrcp   = fitPeak(Bpeak_pts_nrcp, ra, nrcp)
Bfitponl, Bonlrms, Bxpolcp, Bypolcp = fitPeak(Bpeak_pts_onlcp, ra, onlcp)
Bfitponr, Bonrrms, Bxporcp, Byporcp = fitPeak(Bpeak_pts_onrcp, ra, onrcp)

# Print out the ANTENNA TEMPERATURES
print ('Ta_ASLCP = %.3f +- %.3f [K],  Ta_BSLCP = %.3f +- %.3f [K]'   %(max(Afitpsl),  Aslrms,  min(Bfitpsl),  Bslrms))
print ('Ta_ASRCP = %.3f +- %.3f [K],  Ta_BSRCP = %.3f +- %.3f [K]'   %(max(Afitpsr),  Asrrms,  min(Bfitpsr),  Bsrrms))
print ('Ta_ANLCP = %.3f +- %.3f [K],  Ta_BNLCP = %.3f +- %.3f [K]'   %(max(Afitpnl),  Anlrms,  min(Bfitpnl),  Bnlrms))
print ('Ta_ANRCP = %.3f +- %.3f [K],   Ta_BNRCP = %.3f +- %.3f K]'     %(max(Afitpnr),  Anrrms,  min(Bfitpnr),  Bnrrms))
print ('Ta_AONLCP = %.3f +- %.3f [K], Ta_BONLCP = %.3f +- %.3f [K]' %(max(Afitponl), Aonlrms, min(Bfitponl), Bonlrms))
print ('Ta_AONRCP = %.3f +- %.3f [K], Ta_BONRCP = %.3f +- %.3f [K]' %(max(Afitponr), Aonrrms, min(Bfitponr), Bonrrms))


Plot the fits

In [ ]:
def plotFit(ra, drift1, title, peak_pts1,fit1,drift2, peak_pts2,fit2, peak_pts3,fit3, peak_pts4,fit4, lab1,lab2,lab3,lab4,Axp1, Ayp1,Bxp1, Byp1,Axp2, Ayp2,Bxp2, Byp2):
    
    # plot the data
    plt.figure(figsize=[15,5])
    plt.suptitle(title)
    
    ax = plt.subplot(121)
    l1,= plt.plot(ra,drift1, label = 'data')
    plt.plot(Axp1,Ayp1,'y')
    plt.plot(Bxp1,Byp1,'k')
    l3,= plt.plot(ra[peak_pts1],fit1,'r', label=lab1)
    l4,= plt.plot(ra[peak_pts2],fit2,'m', label=lab2)    
    plt.xlabel('RA (deg)')
    plt.ylabel('Ta (K)')
    plt.legend()
    plt.axhline(y=0, color='k', alpha= 0.5)
    plt.grid()
    lim = plt.axis('tight')

    ax = plt.subplot(122)
    l1,= plt.plot(ra,drift2, label = 'data')
    plt.plot(Axp2,Ayp2,'y')
    plt.plot(Bxp2,Byp2,'k')
    l3,= plt.plot(ra[peak_pts3],fit3, 'r',label = lab3)
    l4,= plt.plot(ra[peak_pts4],fit4, 'm', label = lab4)    
    plt.xlabel('RA o(deg)')
    plt.ylabel('Ta (K)')
    plt.axhline(y=0, color='k', alpha= 0.5)
    plt.legend()
    plt.grid()
    lim = plt.axis('tight')

plotFit(ra,slcp,'HPS PEAK FIT',Apeak_pts_slcp, Afitpsl,  srcp, Bpeak_pts_slcp, Bfitpsl, Apeak_pts_srcp, Afitpsr, Bpeak_pts_srcp, Bfitpsr, 'Ta_ASLCP  = %.3f +- %.3f [K]' %(max(Afitpsl), Aslrms),  'Ta_BSLCP = %.3f +- %.3f [K]' %(min(Bfitpsl), Bslrms),  'Ta_ASRCP = %.3f +- %.3f [K]' %(max(Afitpsr), Asrrms), 'Ta_BSRCP = %.3f +- %.3f [K]'  %(min(Bfitpsr), Bsrrms),  Axpslcp, Aypslcp, Bxpslcp, Bypslcp, Axpsrcp, Aypsrcp, Bxpsrcp, Bypsrcp)
plotFit(ra,nlcp,'HPN PEAK FIT',Apeak_pts_nlcp, Afitpnl,  nrcp, Bpeak_pts_nlcp, Bfitpnl, Apeak_pts_nrcp, Afitpnr, Bpeak_pts_nrcp, Bfitpnr, 'Ta_ANLCP  = %.3f +- %.3f [K]' %(max(Afitpnl), Anlrms),  'Ta_BNLCP = %.3f +- %.3f [K]' %(min(Bfitpnl), Bnlrms),  'Ta_ANRCP = %.3f +- %.3f K]' %(max(Afitpnr), Anrrms), 'Ta_BNRCP = %.3f +- %.3f K]'   %(min(Bfitpnr), Bnrrms),   Axpnlcp, Aypnlcp, Bxpnlcp, Bypnlcp, Axpnrcp, Aypnrcp,Bxpnrcp, Bypnrcp)
plotFit(ra,onlcp,'ON PEAK FIT',Apeak_pts_onlcp,Afitponl, onrcp,Bpeak_pts_onlcp,Bfitponl,Apeak_pts_onrcp,Afitponr,Bpeak_pts_onrcp,Bfitponr,'Ta_AONLCP = %.3f +- %.3f [K]' %(max(Afitponl),Aonlrms), 'Ta_BONLCP = %.3f +- %.3f [K]'%(min(Bfitponl),Bonlrms), 'Ta_AONRCP = %.3f +- %.3f [K]' %(max(Afitponr),Aonrrms),'Ta_BONRCP = %.3f +- %.3f [K]' %(min(Bfitponr),Bonrrms), Axpolcp, Aypolcp, Bxpolcp, Bypolcp, Axporcp, Ayporcp,Bxporcp, Byporcp)



## 4.4 <a name="section5.4">Amplitude correction</a>
[back to top](#top)

In [ ]:
def correctAmp(hpn,errhpn,hps,errhps,on,erron):

    ''' Calculate the pointing correction

        Tcorr = Ton x exp[((ln(Thps) - ln(Thpn))**2)/(16*ln(2))]

        Tcorr = Ton x pc

        term1 = ((ln(Thps) - ln(Thpn))**2)
        term2 = 16*ln(2)
        term3 = term1/term2
        pc = exp[term3]
    '''

    # Calculate the pointing correction
    term1 = (np.log(abs(hps)) - np.log(abs(hpn)))**2
    term2 = 16*np.log(2)
    term3 = term1/term2
    pc = np.exp(term3)
            
    # Calculate the corrected antenna temperature
    Tcorr = on * pc

    # Calculate the error in the pointing correction
    if hps < 0.0:
        pf1 = pc * 2.0 * term3 * (1.0/hps)
    else:
        pf1 = pc * 2.0 * term3 * (1.0/abs(hps))
                                
    if hpn < 0.0:
        pf2 = pc * 2.0 * term3 * (-1.0/hpn)
    else:
        pf2 = pc * 2.0 * term3 * (1.0/abs(hpn))
            
    pcerr = np.sqrt((errhpn**2) + (errhps**2)) # err in pc

    # Calculate the error in the corrected antenna temperature
    Tcorrerr = np.sqrt((erron/on)**2 + (pcerr/pc)**2)*Tcorr
            
    return pc, pcerr, Tcorr, Tcorrerr


### Put Link to how we calculate the error propagation


Calculate the corrected amplitude



In [ ]:
# Calculate Pointing correction
Apcl, Apcerrl, ATcorrl, ATcorerrl = correctAmp(max(Afitpnl), Anlrms, max(Afitpsl), Aslrms, max(Afitponl), Aonlrms)
Bpcl, Bpcerrl, BTcorrl, BTcorerrl = correctAmp(min(Bfitpnl), Bnlrms, min(Bfitpsl), Bslrms, min(Bfitponl), Bonlrms)

Apcr, Apcerrr, ATcorrr, ATcorerrr = correctAmp(max(Afitpnr), Anrrms, max(Afitpsr), Asrrms, max(Afitponr), Aonrrms)
Bpcr, Bpcerrr, BTcorrr, BTcorerrr = correctAmp(min(Bfitpnr), Bnrrms, min(Bfitpsr), Bsrrms, min(Bfitponr), Bonrrms)

print ('Ta_ALCP corr = %.4f +- %.4f [K]' %(max(Afitponl), Aonlrms))
print ('Ta_BLCP corr = %.4f +- %.4f [K]' %(min(Bfitponl), Bonlrms))
print ('Ta_ARCP corr = %.4f +- %.4f [K]' %(max(Afitponr), Aonrrms))
print ('Ta_BRCP corr = %.4f +- %.4f [K]\n' %(min(Bfitponr), Bonrrms))

print ('Amplitude correction factor due to pointing error in A BEAM LCP = %.4f +- %f [K] ' %(Apcl, Apcerrl))
print ('Amplitude correction factor due to pointing error in B BEAM LCP = %.4f +- %f [K] ' %(Bpcl, Bpcerrl))
print ('Amplitude correction factor due to pointing error in A BEAM RCP = %.4f +- %f [K] ' %(Apcr, Apcerrr))
print ('Amplitude correction factor due to pointing error in B BEAM RCP = %.4f +- %f [K]\n' %(Bpcr, Bpcerrr))

print ('Ta_ALCP corr = %.4f +- %.4f [K]' %(ATcorrl, ATcorerrl))
print ('Ta_BLCP corr = %.4f +- %.4f [K]' %(BTcorrl, BTcorerrl))
print ('Ta_ARCP corr = %.4f +- %.4f [K]' %(ATcorrr, ATcorerrr))
print ('Ta_BRCP corr = %.4f +- %.4f [K]' %(BTcorrr, BTcorerrr))

# 5. <a name="section6">Calculate the Point source sensitivity</a>
[back to top](#top)

We need a conversion factor for our target source flux. We need to calculate 
the PSS of the calibrator source


For single dish observations, the flux standards defined by Ott et al. (1994) (http://adsabs.harvard.edu/full/1994A&amp;A...284..331O) are still in use (http://adsabs.harvard.edu/abs/1977A%26A....61...99B is an earlier flux scale).  At HartRAO we generally observe three flux calibrators: 3C123, Virgo A and Hydra A.  Most of the sources, with the exception of 3C286 and 3C295 (both too faint to be of much use to us) have shown variability on the time-scale of a decade. So it is generally a good idea to have multiple calibrators.  Three is the minimum number that would enable you to determine which source has undergone intrinsic variation.  

The spectra of the sources are characterised by a frequency dependent expression with three coefficients. 
$$\text{log } S\text{[Jy]} = a + b * \text{log }\nu \text{[MHz]} + c * \text{log}^2 \nu \text{[MHz]}$$

Table 5 of Ott et al. (1994) is shown below:

![Ott table](images/Ott_calibrators.png)

In [ ]:
# Estimate the flux 
def S_ott(a, b, c, nu):
    
    #evaluate the Ott flux polynomial
    return 10**(a + b * np.log10(nu) + c * np.log10(nu)**2)

In [ ]:
nu = drift1.header['CENTFREQ']
HydA = S_ott(4.728, -1.025, 0.0130, nu)  #3C218 in Ott table
print('Flux density of Hydra A at %.3f MHz is %.2f Jy'%(nu, HydA))

Note that this is the total flux of the source.  Each polarisation of the reciever only picks up half of this flux.


### Calculate the PSS

In [ ]:
def calcpss(pc, Tcorr, Tcorerr,flux):
    """
     Calculate PSS
    """
    pss = abs(float(flux)/2.0/Tcorr) 
    psserr = np.sqrt((Tcorerr/Tcorr)**2)*pss
    
    return pss, psserr


In [ ]:
print ('Ta_ALCP corr = %.3f +- %.3f [K]' %(ATcorrl, ATcorerrl))
print ('Ta_BLCP corr = %.3f +- (%.3f) [K]' %(BTcorrl, BTcorerrl))
print ('Ta_ARCP corr = %.3f +- %.3f [K]' %(ATcorrr, ATcorerrr))
print ('Ta_BRCP corr = %.3f +- (%.3f) [K]\n' %(BTcorrr, BTcorerrr))

Apsslcp, Aerrpsslcp = calcpss(Apcl, ATcorrl, ATcorerrl,HydA)
Bpsslcp, Berrpsslcp = calcpss(Bpcl, BTcorrl, BTcorerrl,HydA)
Apssrcp, Aerrpssrcp = calcpss(Apcr, ATcorrr, ATcorerrr,HydA)
Bpssrcp, Berrpssrcp = calcpss(Bpcr, BTcorrr, BTcorerrr,HydA)


print('Point source sensitivity of A BEAM LCP channel is %.3f +- %.3f Jy/K'%(Apsslcp, Aerrpsslcp))
print('Point source sensitivity of B BEAM LCP channel is %.3f +- %.3f Jy/K'%(Bpsslcp, Berrpsslcp))
print('Point source sensitivity of A BEAM RCP channel is %.3f +- %.3f Jy/K'%(Apssrcp, Aerrpssrcp))
print('Point source sensitivity of B BEAM RCP channel is %.3f +- %.3f Jy/K'%(Bpssrcp, Berrpssrcp))


We have successfuly calculated the PSS of the calibrator Hydra A @ 5GHz, we can go 
on to estimate the flux density of our variable source.

Link to source [J1427-4206](5GHz_j1427-4206_tutorial.ipynb) notebook 